In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# This will reload imports before executing code, allowing you to easily change contents of custom scripts
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
import os, sys
sys.path.append('/home/sandm/Notebooks/stay_classification/src/')

# Random trajectory creation

Create a (random) sequence of signaling events, _eg_ GPS measurements, with locationsobscured by noise. 


**Note** the signaling events for a given user, form a set $\mathcal{E} :=  \{e_i = (\mathbf{x}_i, t_i), i=[0,N-1] \; | \; t_{i+1}>t_i\}$

## Configs

There are a few main configs:
* the minimal duration of the stays
* the minimal distance between two adjacent stays
* the rest are just ranges for some stochastics which control the noise 

In [6]:
configs = {
    'time_thresh':1/6,
    'dist_thresh':0.5,
    'noise_min':0.02,
    'noise_max':0.15
}

## Create the trajectories

The trajectory is specified by a time array and a locations array. 

There are a few additional arrays, namely 
* one with the raw trajectory without the noise
    * these are used for evaluating the classification
* lists of the stays as segments defined by the pairs $[(\mathbf{x}_i, t_i)$, $(\mathbf{x}_i, t_j)]$
    * these are what should be identified

In [7]:
from synthetic_data.trajectory import get_stay_segs, get_adjusted_stays
from synthetic_data.trajectory_class import get_rand_traj

In [115]:
time_arr, raw_arr, noise_arr, segments = get_rand_traj(configs)
t_segs, x_segs = get_stay_segs(get_adjusted_stays(segments, time_arr))

## Visualizations

In [54]:
from synthetic_data.plotting import plot_trajectory

In [116]:
plot_trajectory(time_arr, raw_arr, noise_arr, t_segs, x_segs, configs['dist_thresh'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Save the trajectory

### Pickling

In [11]:
from synthetic_data.trajectory_class import pickle_trajectory
from datetime import datetime

In [12]:
stays_tag = int((x_segs.size)/3)
date_tag = datetime.today().strftime('%Y%m%d')
notes = ''
trajectory_tag = f"{date_tag}_trajectory_{stays_tag}stays__{notes}"
path_to_file =  "./testdata/" + trajectory_tag + ".pkl"
print(path_to_file)

./testdata/20200703_trajectory_5stays__.pkl


In [13]:
pickle_trajectory(time_arr, raw_arr, noise_arr, segments, None)

TypeError: expected str, bytes or os.PathLike object, not NoneType

### tabling

In [14]:
from synthetic_data.trajectory import get_stay_indices

In [147]:
true_indices = get_stay_indices(get_adjusted_stays(segments, time_arr), time_arr)
true_labels = np.zeros(time_arr.shape)

for pair in true_indices:
    true_labels[pair[0]:pair[1]+1] = 1

In [16]:
import pandas as pd

In [148]:
df = pd.DataFrame({'ID':time_arr.size*["#111111"], 'time':time_arr, 'x':raw_arr, 'nx':noise_arr, 'stay':np.array(true_labels).astype(bool)})

In [149]:
df.head(10)

,ID,time,x,nx,stay
0,#111111,0.055278,-1.706245,-1.709878,True
1,#111111,0.109722,-1.706245,-1.763388,True
2,#111111,0.195833,-1.706245,-1.730857,True
3,#111111,0.203333,-1.706245,-1.652080,True
4,#111111,0.245000,-1.706245,-1.634114,True
5,#111111,0.315556,-1.706245,-1.700113,True
6,#111111,0.324167,-1.706245,-1.758871,True
7,#111111,0.378611,-1.669235,-1.663362,False
8,#111111,0.381944,-1.663152,-1.736698,False
9,#111111,0.404722,-1.621580,-1.640583,False


In [150]:
fig, ax = plt.subplots(figsize=[20,10])
df.plot(x='time', y='x', marker='', linestyle='--', ax=ax)
df.plot(x='time', y='nx', marker='.', linestyle='-', ax=ax)

# To print the segments
df_cp = df.copy()
df_cp.loc[df_cp['stay']==False,'nx'] =  np.nan
df_cp.loc[df_cp['stay']==False,'x'] =  np.nan
df_cp.plot(x='time', y='x', color='k',marker='o', linestyle='--', dashes=[1,0.5], linewidth=5, ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
path_to_file =  "./testdata/" + trajectory_tag + ".csv"
df.to_csv(path_to_filez, index=False)